### Restart and run all

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

engine = create_engine("sqlite:///c:\\ruby\\expense\\db\\development.sqlite3")
conep = engine.connect()

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'amount':'{:,}','y_amt':'{:,}','yoy_gain':'{:,}',
               'q_amt_c':'{:,}','q_amt_p':'{:,}',
               'aq_amt':'{:,}','ay_amt':'{:,}','acc_gain':'{:,}',
               'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
               'latest_amt_q':'{:,}','previous_amt_q':'{:,}','inc_amt_q':'{:,}',
               'latest_amt_y':'{:,}','previous_amt_y':'{:,}','inc_amt_y':'{:,}',
               'inc_pct':'{:.2f}','inc_pct_q':'{:.2f}','inc_pct_y':'{:.2f}'}
year = 2023
month = 1
today = date.today()
today

datetime.date(2023, 1, 9)

In [2]:
sql = """
SELECT date, amount, C.name AS item, G.name AS kind, strftime("%m", date) as Month 
FROM transactions T 
JOIN categories C ON category_id = C.id 
JOIN groups G ON group_id = G.id 
WHERE strftime('%Y', date) = '2023'
"""
#sql = sql % year
print(sql)


SELECT date, amount, C.name AS item, G.name AS kind, strftime("%m", date) as Month 
FROM transactions T 
JOIN categories C ON category_id = C.id 
JOIN groups G ON group_id = G.id 
WHERE strftime('%Y', date) = '2023'



In [3]:
df_t = pd.read_sql(sql, conep)
df_t.style.format(format_dict)

,date,amount,item,kind,Month
0,2023-01-04,"5,670",Pets,Pets,01
1,2023-01-04,"10,960",Medicine,Medicine,01
2,2023-01-02,"1,670",Food,Food,01
3,2023-01-04,280,Food,Food,01
4,2023-01-03,100,Food,Food,01
5,2023-01-01,"1,200",Fuel,Transportation,01
6,2023-01-01,120,Food,Food,01
7,2023-01-05,920,Pets,Pets,01
8,2023-01-04,100,Merit Making,Others,01
9,2023-01-09,"3,590",Plum,Shelter,01


In [4]:
float_formatter = "฿{:,.2f}"
ttl_exp = df_t.amount.sum()
float_formatter.format(ttl_exp)

'฿34,710.00'

In [5]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price,
       (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, 
       buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price,
       (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, 
       buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023
ORDER BY sells.date DESC, name


In [6]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,JMART,2023,1,2022,12,42.000000,40.000000,2.000000,3000,126000.000000,120000.000000,6000.000000,5.000000,5455.130000,DTD
1,SINGER,2023,1,2023,1,29.250000,27.500000,1.750000,3600,105300.000000,99000.000000,6300.000000,6.360000,5847.490000,DTD
2,GFPT,2023,1,2022,12,13.100000,12.500000,0.600000,7500,98250.000000,93750.000000,4500.000000,4.800000,4074.740000,DTD


In [7]:
sells_df.groupby(['name'])[['gross','net']].sum()

,gross,net
name,,
GFPT,4500.0,4074.74
JMART,6000.0,5455.13
SINGER,6300.0,5847.49


In [8]:
float_formatter = "฿{:,.2f}"
ttl_net = sells_df.net.sum()
float_formatter.format(ttl_net)

'฿15,377.36'

In [9]:
net_flow = ttl_net - ttl_exp
ary = [ttl_net, ttl_exp, net_flow]
ary

[15377.359999999999, 34710, -19332.64]